In [1]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
from tqdm import tqdm
import torch
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

name = "m3rg-iitd/matscibert"  # "allenai/scibert_scivocab_cased"
df = pd.read_parquet('/mnt/hdd1/LaMDa/mp_3d_2020_gpt_narratives.parquet')
tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForMultipleChoice.from_pretrained(name)


/home/yj/anaconda3/envs/clamp_p/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at m3rg-iitd/matscibert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model = model.cuda()
answer_list = []
for i, row in tqdm(df.iterrows()):
    try:
        row['atoms'].pop('coords')
        row['atoms'].pop('props')
        prompt = 'Which of the following statements is most correct regarding the given material?' + str(row['atoms'])
        inputs = tokenizer([[prompt, candidate] for candidate in row['structure_question_list']], return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0).cuda()

        outputs = model(**{k: v.unsqueeze(0).cuda() for k, v in inputs.items()}, labels=labels)
        logits = outputs.logits
        answer_list.append(logits.argmax().item())
    except:
        answer_list.append(-1)

126335it [1:28:19, 23.84it/s]


In [3]:
# count -1 
sum([i == 0 for i in answer_list])/len(answer_list)

0.081212648909645

In [4]:
model = model.cuda()
comp_answer_list = []
for i, row in tqdm(df.iterrows()):
    try:
        # row['atoms'].pop('coords')
        # row['atoms'].pop('props')
        prompt = 'Which of the following statements is most correct regarding the given material?' + str(row['atoms'])
        inputs = tokenizer([[prompt, candidate] for candidate in row['composition_question_list']], return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0).cuda()

        outputs = model(**{k: v.unsqueeze(0).cuda() for k, v in inputs.items()}, labels=labels)
        logits = outputs.logits
        comp_answer_list.append(logits.argmax().item())
    except:
        comp_answer_list.append(-1)

0it [00:00, ?it/s]

126335it [2:16:33, 15.42it/s]


In [5]:
sum([i == 0 for i in comp_answer_list])/len(comp_answer_list), sum([i == -1 for i in comp_answer_list])

(0.015411406181976492, 11074)

In [6]:
model = model.cuda()
oxide_answer_list = []
for i, row in tqdm(df.iterrows()):
    try:
        # row['atoms'].pop('coords')
        # row['atoms'].pop('props')
        prompt = 'Which of the following statements is most correct regarding the given material?' + str(row['atoms'])
        inputs = tokenizer([[prompt, candidate] for candidate in row['oxide_question_list']], return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0).cuda()

        outputs = model(**{k: v.unsqueeze(0).cuda() for k, v in inputs.items()}, labels=labels)
        logits = outputs.logits
        oxide_answer_list.append(logits.argmax().item())
    except:
        oxide_answer_list.append(-1)

0it [00:00, ?it/s]

126335it [1:15:56, 27.72it/s]


In [7]:
sum([i == 0 for i in oxide_answer_list])/len(oxide_answer_list), sum([i == -1 for i in oxide_answer_list])

(0.00508964261685202, 7391)

In [8]:
model = model.cuda()
comp_struc_answer_list = []
for i, row in tqdm(df.iterrows()):
    try:
        # row['atoms'].pop('coords')
        # row['atoms'].pop('props')
        prompt = 'Which of the following statements is most correct regarding the given material?' + str(row['atoms'])
        inputs = tokenizer([[prompt, candidate] for candidate in row['comp_struc_question_list']], return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0).cuda()

        outputs = model(**{k: v.unsqueeze(0).cuda() for k, v in inputs.items()}, labels=labels)
        logits = outputs.logits
        comp_struc_answer_list.append(logits.argmax().item())
    except:
        comp_struc_answer_list.append(-1)

0it [00:00, ?it/s]

126335it [1:27:11, 24.15it/s]


In [9]:
sum([i == 0 for i in comp_struc_answer_list])/len(comp_struc_answer_list), sum([i == -1 for i in comp_struc_answer_list])

(0.09204891756045434, 7938)